In [ ]:
import requests
from bs4 import BeautifulSoup
import bs4
import pandas as pd
from utils import *
import os
import gradio as gr
from copy import deepcopy
import random
#lxml html5lib 

In [ ]:
if not os.path.exists('data\companion_plants.csv'):
    companion = Plant_Table(r"https://en.wikipedia.org/wiki/List_of_companion_plants", ['common name','scientific name','helps','helped by','attracts','repels/traps','avoid'], del_df=[5, 6], del_cols=['comments'], merges=[], unmerge=[])
    weeds = Plant_Table(r"https://en.wikipedia.org/wiki/List_of_beneficial_weeds", ['common name','scientific name','helps','attracts/hosts'], del_df=[], del_cols=['medicinal','edibility','comments'], merges=[['repels/traps','repels','traps']], unmerge=[])
    df = weeds + companion
    df.save()
    del companion, weeds, df

df = Plant_Table()


In [ ]:
# gradio
example_plants = random.sample(df.get_plant_names(), len(df.get_plant_names()))

def get_compatible_plants(plant_name, strict='strictly_intercompatible'):
    compatible = df.get_compatible_groups(plant_name, strict=strict)
    if len(compatible) == 0:
        return "No compatible plants found."
    else:
        for i, item in enumerate(compatible):
            compatible[i] = ', '.join(item)
        compatible = '\n\n'.join(compatible)

    return compatible

## Interface
# plant_demo = gr.Interface(
#     fn=get_compatible_plants, 
#     inputs="text", 
#     outputs="text", 
#     title="Companion Planting", 
#     description="Enter a plant name to see groups of plants which are compatible with it.", 
#     examples=[[example] for example in example_plants],
#     allow_flagging='never',
#     examples_per_page=len(example_plants),
# )

#Blocks Interface (more customization)
with gr.Blocks(title="Companion Planting") as plant_demo:
    with gr.Row():
        with gr.Column():
            plant_name = gr.Textbox(label="Plant Name")
            input_btn = gr.Button(label="Get Compatible Plants")
        with gr.Column():
            output = gr.Textbox(label="Compatible Plants")

    input_btn.click(get_compatible_plants, plant_name, outputs=output)
    examples = gr.Examples(examples=[[example] for example in example_plants], label="Examples", inputs=plant_name, examples_per_page=len(example_plants))

# Tabbed Interface

            

plant_demo.launch(share=False, inbrowser=True, inline=False)